# 09. 순환 신경망(Recurrent Neural Network)

## 6) RNN을 이용한 텍스트 생성(Text Generation using RNN)

### 1) 데이터에 대한 이해와 전처리

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
          그의 말이 법이다\n
          가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [6]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [7]:
max_len = max(len(l) for l in sequences) 
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인하였습니다. 

전체 샘플의 길이를 6으로 패딩합니다.

In [8]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [9]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


리스트의 마지막 값을 제외하고 저장한 것은 X, 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됩니다.

In [10]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [11]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [12]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [13]:
y = to_categorical(y, num_classes=vocab_size)

RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행합니다.

In [14]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2) 모델 설계하기

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 5s - loss: 2.4528 - accuracy: 0.2727
Epoch 2/200
1/1 - 0s - loss: 2.4396 - accuracy: 0.2727
Epoch 3/200
1/1 - 0s - loss: 2.4263 - accuracy: 0.2727
Epoch 4/200
1/1 - 0s - loss: 2.4126 - accuracy: 0.2727
Epoch 5/200
1/1 - 0s - loss: 2.3985 - accuracy: 0.3636
Epoch 6/200
1/1 - 0s - loss: 2.3839 - accuracy: 0.3636
Epoch 7/200
1/1 - 0s - loss: 2.3688 - accuracy: 0.3636
Epoch 8/200
1/1 - 0s - loss: 2.3530 - accuracy: 0.3636
Epoch 9/200
1/1 - 0s - loss: 2.3366 - accuracy: 0.3636
Epoch 10/200
1/1 - 1s - loss: 2.3194 - accuracy: 0.3636
Epoch 11/200
1/1 - 0s - loss: 2.3014 - accuracy: 0.3636
Epoch 12/200
1/1 - 0s - loss: 2.2825 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.2628 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.2421 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2206 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.1982 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.1749 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.1508 - accuracy: 0.3636
E

Epoch 147/200
1/1 - 0s - loss: 0.2356 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2313 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2270 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.2227 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.2186 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.2146 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.2106 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.2067 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.2029 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.1991 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1955 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1919 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1884 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1849 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1816 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1783 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1750 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력해봅시다.

In [16]:
def sentence_generation(model, tokenizer, current_word, n): 
    # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [17]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [18]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [19]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


### 2. LSTM을 이용하여 텍스트 생성하기

#### 1) 데이터에 대한 이해와 전처리

In [20]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('datasets/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [21]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [22]:
print(df['headline'].isnull().values.any())
# Null 값은 별도로 없는 것으로 보입니다.

False


headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장해보도록 하겠습니다.

In [23]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values)) 
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [24]:
print('노이즈값 제거 전 샘플의 개수 : {}'.format(len(headline)))
headline = [word for word in headline if word != "Unknown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 전 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [25]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

**구두점 제거**와 단어의 **소문자화**를 수행합니다. 

In [26]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preporcessed_headline = [repreprocessing(x) for x in headline]
preporcessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

단어 집합(vocabulary)을 만들고 크기를 확인합니다.

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preporcessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


정수 인코딩을 진행하는 동시에 하나의 문장을 여러 줄로 분해하여 훈련 데이터를 구성합니다.

In [28]:
sequences = list()

for sentence in preporcessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만듭니다.

In [29]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): 
    # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [30]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩하겠습니다.

In [31]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [32]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [33]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [34]:
print(y[:3])

[ 269  371 1115]


레이블 데이터 y에 대해서 원-핫 인코딩을 수행하겠습니다.

In [36]:
y = to_categorical(y, num_classes=vocab_size)

#### 2) 모델 설계하기

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 11s - loss: 7.6365 - accuracy: 0.0306
Epoch 2/200
244/244 - 3s - loss: 7.1099 - accuracy: 0.0299
Epoch 3/200
244/244 - 3s - loss: 6.9723 - accuracy: 0.0351
Epoch 4/200
244/244 - 2s - loss: 6.8415 - accuracy: 0.0425
Epoch 5/200
244/244 - 2s - loss: 6.6893 - accuracy: 0.0447
Epoch 6/200
244/244 - 1s - loss: 6.5201 - accuracy: 0.0495
Epoch 7/200
244/244 - 3s - loss: 6.3341 - accuracy: 0.0519
Epoch 8/200
244/244 - 5s - loss: 6.1409 - accuracy: 0.0557
Epoch 9/200
244/244 - 3s - loss: 5.9582 - accuracy: 0.0636
Epoch 10/200
244/244 - 3s - loss: 5.7774 - accuracy: 0.0655
Epoch 11/200
244/244 - 4s - loss: 5.6076 - accuracy: 0.0719
Epoch 12/200
244/244 - 3s - loss: 5.4476 - accuracy: 0.0751
Epoch 13/200
244/244 - 3s - loss: 5.2932 - accuracy: 0.0813
Epoch 14/200
244/244 - 4s - loss: 5.1454 - accuracy: 0.0865
Epoch 15/200
244/244 - 1s - loss: 5.0031 - accuracy: 0.0939
Epoch 16/200
244/244 - 2s - loss: 4.8653 - accuracy: 0.1055
Epoch 17/200
244/244 - 1s - loss: 4.7339 - accur

244/244 - 1s - loss: 0.3750 - accuracy: 0.9116
Epoch 138/200
244/244 - 1s - loss: 0.3696 - accuracy: 0.9121
Epoch 139/200
244/244 - 1s - loss: 0.3680 - accuracy: 0.9132
Epoch 140/200
244/244 - 1s - loss: 0.3739 - accuracy: 0.9107
Epoch 141/200
244/244 - 1s - loss: 0.3632 - accuracy: 0.9136
Epoch 142/200
244/244 - 1s - loss: 0.3529 - accuracy: 0.9132
Epoch 143/200
244/244 - 1s - loss: 0.3500 - accuracy: 0.9149
Epoch 144/200
244/244 - 1s - loss: 0.3445 - accuracy: 0.9134
Epoch 145/200
244/244 - 1s - loss: 0.3394 - accuracy: 0.9166
Epoch 146/200
244/244 - 1s - loss: 0.3367 - accuracy: 0.9144
Epoch 147/200
244/244 - 1s - loss: 0.3341 - accuracy: 0.9146
Epoch 148/200
244/244 - 1s - loss: 0.3329 - accuracy: 0.9143
Epoch 149/200
244/244 - 1s - loss: 0.3303 - accuracy: 0.9162
Epoch 150/200
244/244 - 1s - loss: 0.3262 - accuracy: 0.9143
Epoch 151/200
244/244 - 1s - loss: 0.3232 - accuracy: 0.9149
Epoch 152/200
244/244 - 1s - loss: 0.3216 - accuracy: 0.9130
Epoch 153/200
244/244 - 1s - loss: 0.3

문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

In [38]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

임의의 단어 'i'에 대해서 10개의 단어를 추가 생성해봅시다.

In [39]:
print(sentence_generation(model, tokenizer, 'i', 10))

i want to be rich and im not sorry think say


In [40]:
print(sentence_generation(model, tokenizer, 'how', 10))

how to make a crossword puzzle tells fire theyre the art
